In [37]:
import re
import os
import cv2
import time
import codecs
import random
import shutil
import requests
import pyautogui
import pywinauto
from pywinauto import Application
import configparser
from pathlib import Path
from chardet import detect
from selenium import webdriver
from matplotlib import pyplot as plt
# from urllib.parse import unquote, urlparse
from bs4 import BeautifulSoup as BS
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select

In [3]:
def _load_config():
    config_path = "./config.ini"
    with open(config_path, "rb") as ef:
        config_encoding = detect(ef.read())["encoding"]
    config = configparser.ConfigParser()
    config.read_file(codecs.open(config_path, "r", config_encoding))
    return config

In [4]:
def _requests_retry_session(config, status_forcelist=(500, 502, 504), session=None):
    session = requests.session()
    headers = {"user-agent": config["Requests_header"]["user-agent"]}
    session.headers.update(headers)
    return session

In [5]:
def mkdirs(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [223]:
config = _load_config()

In [7]:
home_path = str(Path.home()).replace("\\", "/")

In [8]:
Path().absolute()

WindowsPath('D:/python/tpech')

In [52]:
# dir_cache = "./cache"
# shutil.rmtree(dir_cache, ignore_errors=True)
# mkdirs(dir_cache)

In [153]:
def selenium_setting():
    # Selenium setting
    chrome_opt = Options()
#     chrome_opt.headless = True
    chrome_opt.add_argument("--disable-gpu")
    chrome_opt.add_argument("--mute-audio")
    chrome_opt.add_argument("--ignore-gpu-blocklist")
    chrome_opt.add_argument("--no-default-browser-check")
    chrome_opt.add_argument("--disable-default-apps")
    chrome_opt.add_argument("--disable-extensions")
    chrome_opt.add_argument("--hide-scrollbars")
#     chrome_opt.add_argument("disk-cache-dir={}/{}".format(Path().absolute(), dir_cache))
    chrome_opt.add_experimental_option("excludeSwitches", ["enable-logging"])
    chrome_opt.binary_location = config["Chrome_Canary"]["CHROME_PATH"].format(home_path)
    return chrome_opt

In [154]:
chrome_opt = selenium_setting()

In [228]:
driver.quit()

In [219]:
dir_downloads = "{}/downloads/page_files".format(Path.home())
page_path = "{}/downloads/page.html".format(Path.home())

In [221]:
shutil.rmtree(dir_downloads, ignore_errors=True)
if os.path.exists(page_path):
    Path(page_path).unlink()

In [225]:
url = config["reg_url"]["url"]
url

'https://webreg.tpech.gov.tw/RegOnline1_2.aspx?ZCode=P&DeptCode=0100&deptname=家庭醫學科'

In [226]:
driver = webdriver.Chrome(
            executable_path=config["Chrome_Canary"]["CHROMEDRIVER_PATH"],
            options=chrome_opt
        )

In [227]:
driver.get(url)

In [184]:
soup = BS(driver.page_source, "lxml")

In [185]:
doc_name = soup.find_all("input", {"name":"RadioDoctor"})[0].parent.a.text

In [186]:
driver.find_element_by_link_text(doc_name).click()

In [229]:
id_no = config["reg_info"]["id"]
birth_y = config["reg_info"]["birth_year"]
birth_m = config["reg_info"]["birth_month"]
birth_d = config["reg_info"]["birth_day"]
phone = config["reg_info"]["phone"]

In [187]:
driver.find_element_by_name("no").send_keys(id_no)

In [188]:
# Select(driver.find_element_by_name("yeartype")).select_by_value("民國")

In [189]:
Select(driver.find_element_by_name("y1")).select_by_value(birth_y)

In [190]:
Select(driver.find_element_by_name("m1")).select_by_value(birth_m)

In [191]:
Select(driver.find_element_by_name("d1")).select_by_value(birth_d)

In [192]:
driver.find_element_by_id("rbAgree").click()

In [193]:
driver.find_element_by_name("txtCellPhone").send_keys(phone)

In [197]:
verify = driver.find_element_by_xpath("//img[@src='ValidateCode.aspx']")

In [200]:
pid = pywinauto.findwindows.find_element(title_re="網路掛號").process_id
pid

12092

In [201]:
app = Application().connect(process=pid)

In [202]:
win = app.window()

In [203]:
win.set_focus()
time.sleep(0.5)
pyautogui.hotkey("ctrl", "s")
time.sleep(1)
pyautogui.typewrite("page.html")
pyautogui.hotkey("enter")

In [213]:
verify_img = "{}/ValidateCode.aspx".format(dir_downloads)
verify_img

'C:\\Users\\Quentin/downloads/page_files/ValidateCode.aspx'

In [214]:
os.path.exists(verify_img)

True

In [216]:
while not os.path.exists(verify_img):
    time.sleep(0.1)